In [1]:
%load_ext rpy2.ipython

In [2]:
%%R
set.seed(88888888) # maximum luck

library(DESeq2)
library(plotly)
library(ggplot2)
library(viridis)
library(magrittr)
library(pheatmap)
library(DescTools)
library(pdfCluster)
library(RColorBrewer)
library(SummarizedExperiment)
library(caret)
library(class)
library(htmlwidgets)

Loading required package: S4Vectors
Loading required package: stats4
Loading required package: BiocGenerics

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which.max, which.min


Attaching package: ‘S4Vectors’

The following object is masked from ‘package:utils’:

    findMatches

The following objects are masked from ‘package:base’:

    expand.grid, I, unname

Loading required package: IRanges
Loading required package: GenomicRanges
Loading required package: GenomeInfoDb
Loading require

In [3]:
%%R
OUT_DIR <- "/mnt/home/yuankeji/RanceLab/reticula_new/reticula/data/tcdd/learning_curve_analysis/output/"
N_FOLDS <- 10

In [4]:
%%R
vst.counts <- readRDS(paste(OUT_DIR, "vst_counts030_time_cross_70.Rds", sep = ""))
print(vst.counts)

class: DESeqTransform 
dim: 9297 94 
metadata(1): version
assays(1): ''
rownames(9297): ENSMUSG00000000001 ENSMUSG00000000028 ...
  ENSMUSG00000118462 ENSMUSG00000118541
rowData names(4): baseMean baseVar allZero dispFit
colnames(94): SRR7817613 SRR7817695 ... SRR1636675 SRR7817701
colData names(3): Sample Dose sizeFactor


In [5]:
%%R
sample <- colData(vst.counts)
# print(sample)

In [6]:
%%R
vst.count.mtx <-
   vst.counts %>% SummarizedExperiment::assay() %>% as.data.frame()
print(length(vst.count.mtx))
 head(vst.count.mtx)

[1] 94
                   SRR7817613 SRR7817695 SRR7817698 SRR1636593 SRR7817637
ENSMUSG00000000001  15.657278  15.830531  15.521473  16.331873  15.737249
ENSMUSG00000000028   7.896070   7.084628   7.449454   8.330852   7.604541
ENSMUSG00000000049  20.359191  19.954578  20.186495  20.383123  20.399895
ENSMUSG00000000058   8.448876   8.629453   8.447006   8.852873   9.186141
ENSMUSG00000000085  10.697905  10.738284  10.429383  10.398967  11.373079
ENSMUSG00000000088  16.775240  16.685460  16.624890  16.029662  16.371583
                   SRR7817686 SRR7817614 SRR7817639 SRR7817616 SRR7817696
ENSMUSG00000000001  15.890296  15.686193  15.579970  15.651784  15.858693
ENSMUSG00000000028   7.379612   7.828167   7.688808   7.672576   7.224958
ENSMUSG00000000049  20.095075  20.350239  20.122341  20.257740  19.950017
ENSMUSG00000000058   8.610605   8.764256   8.676282   9.046821   8.551272
ENSMUSG00000000085  11.150278  10.871795  11.004445  10.275517  11.311758
ENSMUSG00000000088  16.258154  

ENSMUSG00000000085  11.331639  11.028541  10.835747  10.713323  10.627565
ENSMUSG00000000088  15.695139  16.478049  16.566711  16.154230  15.212206
                   SRR7817666 SRR7817657 SRR7817693 SRR7817620 SRR7817682
ENSMUSG00000000001  16.795675  15.550124  16.559808  16.406026  17.167234
ENSMUSG00000000028   8.477779   8.550708   8.642699   9.414371   7.879733
ENSMUSG00000000049  17.250998  18.068875  17.597813  17.698493  17.202836
ENSMUSG00000000058  10.464612   9.819166   9.994143   9.895659  10.299547
ENSMUSG00000000085  11.294266  11.181099  10.651636  11.132570  11.228619
ENSMUSG00000000088  15.951998  16.175449  16.824154  16.567949  15.752643
                   SRR1636672 SRR7817678 SRR3593577 SRR7817633 SRR7817704
ENSMUSG00000000001  16.513362  16.911296  15.221189  16.521989  17.014506
ENSMUSG00000000028   7.853231   8.337375   8.377550   8.387055   8.296182
ENSMUSG00000000049  19.718863  17.226018  19.420771  17.383510  17.473070
ENSMUSG00000000058  10.631405   9.9282

In [7]:
%%R
tcdd_dose_detail.vec <-
   readRDS(paste(OUT_DIR, "tcdd_dose_detail_vec030_time_cross_70.Rds", sep = ""))
print(tcdd_dose_detail.vec)

 [1]  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
[26]  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 30 30 30
[51] 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30
[76] 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30


In [8]:
%%R
rxn2ensembls.nls <-
   readRDS(paste(OUT_DIR, "rxn2ensembls_nls030_time_cross_70.Rds", sep = ""))
print(length(rxn2ensembls.nls))
print(rxn2ensembls.nls)

[1] 9825
$`R-MMU-170666`
 [1] "ENSMUSG00000000001" "ENSMUSG00000005580" "ENSMUSG00000020431"
 [4] "ENSMUSG00000020654" "ENSMUSG00000021536" "ENSMUSG00000022220"
 [7] "ENSMUSG00000022376" "ENSMUSG00000022840" "ENSMUSG00000022994"
[10] "ENSMUSG00000024524" "ENSMUSG00000028777" "ENSMUSG00000031659"
[13] "ENSMUSG00000032562" "ENSMUSG00000057614"

$`R-MMU-170671`
 [1] "ENSMUSG00000000001" "ENSMUSG00000005580" "ENSMUSG00000020431"
 [4] "ENSMUSG00000020654" "ENSMUSG00000021536" "ENSMUSG00000022220"
 [7] "ENSMUSG00000022376" "ENSMUSG00000022840" "ENSMUSG00000022994"
[10] "ENSMUSG00000024524" "ENSMUSG00000028777" "ENSMUSG00000031659"
[13] "ENSMUSG00000032562" "ENSMUSG00000057614"

$`R-MMU-170674`
 [1] "ENSMUSG00000000001" "ENSMUSG00000005580" "ENSMUSG00000020431"
 [4] "ENSMUSG00000020654" "ENSMUSG00000021536" "ENSMUSG00000022220"
 [7] "ENSMUSG00000022376" "ENSMUSG00000022840" "ENSMUSG00000022994"
[10] "ENSMUSG00000024524" "ENSMUSG00000028777" "ENSMUSG00000031659"
[13] "ENSMUSG00000032562" "ENSM

 [46] "ENSMUSG00000028777" "ENSMUSG00000028950" "ENSMUSG00000028971"
 [49] "ENSMUSG00000029064" "ENSMUSG00000029072" "ENSMUSG00000029236"
 [52] "ENSMUSG00000029371" "ENSMUSG00000029372" "ENSMUSG00000029373"
 [55] "ENSMUSG00000029379" "ENSMUSG00000029380" "ENSMUSG00000029417"
 [58] "ENSMUSG00000029530" "ENSMUSG00000029663" "ENSMUSG00000029713"
 [61] "ENSMUSG00000029819" "ENSMUSG00000031778" "ENSMUSG00000031861"
 [64] "ENSMUSG00000031980" "ENSMUSG00000032192" "ENSMUSG00000032562"
 [67] "ENSMUSG00000032766" "ENSMUSG00000033717" "ENSMUSG00000034117"
 [70] "ENSMUSG00000034855" "ENSMUSG00000035042" "ENSMUSG00000035383"
 [73] "ENSMUSG00000035431" "ENSMUSG00000035448" "ENSMUSG00000036353"
 [76] "ENSMUSG00000036362" "ENSMUSG00000036381" "ENSMUSG00000036402"
 [79] "ENSMUSG00000036437" "ENSMUSG00000036832" "ENSMUSG00000037010"
 [82] "ENSMUSG00000037014" "ENSMUSG00000037140" "ENSMUSG00000037346"
 [85] "ENSMUSG00000037944" "ENSMUSG00000038607" "ENSMUSG00000038668"
 [88] "ENSMUSG00000038811" "ENSMUS

 [4] "ENSMUSG00000019775" "ENSMUSG00000020599" "ENSMUSG00000021219"
 [7] "ENSMUSG00000024186" "ENSMUSG00000026357" "ENSMUSG00000026358"
[10] "ENSMUSG00000026475" "ENSMUSG00000026527" "ENSMUSG00000026678"
[13] "ENSMUSG00000028777" "ENSMUSG00000029101" "ENSMUSG00000032562"
[16] "ENSMUSG00000038530" "ENSMUSG00000042671" "ENSMUSG00000051079"
[19] "ENSMUSG00000052087" "ENSMUSG00000057614" "ENSMUSG00000059810"

$`R-MMU-8982019`
 [1] "ENSMUSG00000000001" "ENSMUSG00000002458" "ENSMUSG00000002459"
 [4] "ENSMUSG00000019775" "ENSMUSG00000020599" "ENSMUSG00000021219"
 [7] "ENSMUSG00000024186" "ENSMUSG00000026357" "ENSMUSG00000026358"
[10] "ENSMUSG00000026475" "ENSMUSG00000026527" "ENSMUSG00000026678"
[13] "ENSMUSG00000028777" "ENSMUSG00000029101" "ENSMUSG00000032562"
[16] "ENSMUSG00000038530" "ENSMUSG00000042671" "ENSMUSG00000051079"
[19] "ENSMUSG00000052087" "ENSMUSG00000057614" "ENSMUSG00000059810"

$`R-MMU-8982020`
 [1] "ENSMUSG00000000001" "ENSMUSG00000002458" "ENSMUSG00000002459"
 [4] "ENSMUS

 [1] "ENSMUSG00000000085" "ENSMUSG00000015120" "ENSMUSG00000018537"
 [4] "ENSMUSG00000024325" "ENSMUSG00000025577" "ENSMUSG00000025578"
 [7] "ENSMUSG00000026021" "ENSMUSG00000026484" "ENSMUSG00000026739"
[10] "ENSMUSG00000028796" "ENSMUSG00000037652" "ENSMUSG00000039989"
[13] "ENSMUSG00000040669"

$`R-MMU-4570463`
 [1] "ENSMUSG00000000085" "ENSMUSG00000015120" "ENSMUSG00000018537"
 [4] "ENSMUSG00000020265" "ENSMUSG00000020738" "ENSMUSG00000024325"
 [7] "ENSMUSG00000025577" "ENSMUSG00000025578" "ENSMUSG00000026484"
[10] "ENSMUSG00000026739" "ENSMUSG00000028796" "ENSMUSG00000031347"
[13] "ENSMUSG00000037652" "ENSMUSG00000039989" "ENSMUSG00000040669"

$`R-MMU-4570499`
 [1] "ENSMUSG00000000085" "ENSMUSG00000015120" "ENSMUSG00000018537"
 [4] "ENSMUSG00000020738" "ENSMUSG00000021546" "ENSMUSG00000024325"
 [7] "ENSMUSG00000025577" "ENSMUSG00000025578" "ENSMUSG00000026484"
[10] "ENSMUSG00000026739" "ENSMUSG00000028796" "ENSMUSG00000037652"
[13] "ENSMUSG00000039989" "ENSMUSG00000040669"

$`R-MM

 [1] "ENSMUSG00000000126" "ENSMUSG00000009900" "ENSMUSG00000010797"
 [4] "ENSMUSG00000012282" "ENSMUSG00000015957" "ENSMUSG00000018486"
 [7] "ENSMUSG00000021994" "ENSMUSG00000022382" "ENSMUSG00000022996"
[10] "ENSMUSG00000022997" "ENSMUSG00000026167" "ENSMUSG00000027840"
[13] "ENSMUSG00000028173" "ENSMUSG00000029671" "ENSMUSG00000030093"
[16] "ENSMUSG00000030170" "ENSMUSG00000033227" "ENSMUSG00000036856"
[19] "ENSMUSG00000036961"

$`R-MMU-3781943`
 [1] "ENSMUSG00000000126" "ENSMUSG00000009900" "ENSMUSG00000010797"
 [4] "ENSMUSG00000012282" "ENSMUSG00000015957" "ENSMUSG00000018486"
 [7] "ENSMUSG00000021994" "ENSMUSG00000022382" "ENSMUSG00000022996"
[10] "ENSMUSG00000022997" "ENSMUSG00000026167" "ENSMUSG00000027840"
[13] "ENSMUSG00000028173" "ENSMUSG00000029671" "ENSMUSG00000030093"
[16] "ENSMUSG00000030170" "ENSMUSG00000033227" "ENSMUSG00000036856"
[19] "ENSMUSG00000036961"

$`R-MMU-205328`
[1] "ENSMUSG00000000127" "ENSMUSG00000005672" "ENSMUSG00000019966"
[4] "ENSMUSG00000032113" "ENSM

...

[1] "ENSMUSG00000024515" "ENSMUSG00000024563" "ENSMUSG00000027660"
[4] "ENSMUSG00000029050" "ENSMUSG00000029478" "ENSMUSG00000032402"

$`R-MMU-173488`
[1] "ENSMUSG00000024515" "ENSMUSG00000024563" "ENSMUSG00000032402"
[4] "ENSMUSG00000063065" "ENSMUSG00000063358"

$`R-MMU-2031355`
[1] "ENSMUSG00000024515" "ENSMUSG00000024563" "ENSMUSG00000027803"
[4] "ENSMUSG00000032402"

$`R-MMU-2106579`
[1] "ENSMUSG00000024515" "ENSMUSG00000024563" "ENSMUSG00000027803"
[4] "ENSMUSG00000032402"

$`R-MMU-2176491`
[1] "ENSMUSG00000024515" "ENSMUSG00000024563" "ENSMUSG00000024589"
[4] "ENSMUSG00000032402"

$`R-MMU-2186607`
[1] "ENSMUSG00000024515" "ENSMUSG00000024563" "ENSMUSG00000028800"
[4] "ENSMUSG00000032402" "ENSMUSG00000047407" "ENSMUSG00000062175"

$`R-MMU-2186643`
[1] "ENSMUSG00000024515" "ENSMUSG00000024563" "ENSMUSG00000024947"
[4] "ENSMUSG00000032402"

$`R-MMU-2187325`
[1] "ENSMUSG00000024515" "ENSMUSG00000024563" "ENSMUSG00000026496"
[4] "ENSMUSG00000032402"

$`R-MMU-2187330`
[1] "ENSMUSG0000

[4] "ENSMUSG00000034781" "ENSMUSG00000034792"

$`R-NUL-428715`
[1] "ENSMUSG00000024639" "ENSMUSG00000029064" "ENSMUSG00000043004"

$`R-MMU-977333`
[1] "ENSMUSG00000024640"

$`R-MMU-1247910`
[1] "ENSMUSG00000024644"

$`R-MMU-9753632`
[1] "ENSMUSG00000024644"

$`R-MMU-198845`
[1] "ENSMUSG00000024646"

$`R-MMU-561041`
[1] "ENSMUSG00000024650" "ENSMUSG00000063796" "ENSMUSG00000067144"

$`R-MMU-5696365`
[1] "ENSMUSG00000024654"

$`R-MMU-444476`
[1] "ENSMUSG00000024659" "ENSMUSG00000042770" "ENSMUSG00000052270"
[4] "ENSMUSG00000060701" "ENSMUSG00000079700"

$`R-MMU-1562626`
[1] "ENSMUSG00000024661" "ENSMUSG00000082062"

$`R-MMU-2046084`
[1] "ENSMUSG00000024665"

$`R-MMU-2046096`
[1] "ENSMUSG00000024665"

$`R-MMU-2046097`
[1] "ENSMUSG00000024665"

$`R-MMU-3000120`
[1] "ENSMUSG00000024682"

$`R-MMU-8868402`
[1] "ENSMUSG00000024687"

$`R-MMU-196964`
[1] "ENSMUSG00000024712"

$`R-MMU-187020`
[1] "ENSMUSG00000024713" "ENSMUSG00000027859" "ENSMUSG00000030513"
[4] "ENSMUSG00000030530"

$`R-MMU-8876

[1] "ENSMUSG00000024990" "ENSMUSG00000032327" "ENSMUSG00000061808"

$`R-MMU-508040`
[1] "ENSMUSG00000025007"

$`R-MMU-2045911`
[1] "ENSMUSG00000025017" "ENSMUSG00000039936" "ENSMUSG00000041417"

$`R-MMU-8985822`
[1] "ENSMUSG00000025020" "ENSMUSG00000066877"

$`R-MMU-8985829`
[1] "ENSMUSG00000025020"

$`R-MMU-9010235`
[1] "ENSMUSG00000025020" "ENSMUSG00000051379"

$`R-MMU-5624130`
[1] "ENSMUSG00000025035" "ENSMUSG00000032558" "ENSMUSG00000046562"

$`R-MMU-5624133`
[1] "ENSMUSG00000025035" "ENSMUSG00000032558" "ENSMUSG00000046562"

$`R-MMU-5638004`
[1] "ENSMUSG00000025035" "ENSMUSG00000046562" "ENSMUSG00000060090"

$`R-MMU-5638006`
[1] "ENSMUSG00000025035" "ENSMUSG00000046562" "ENSMUSG00000060090"

$`R-MMU-5638007`
[1] "ENSMUSG00000025035" "ENSMUSG00000046562" "ENSMUSG00000060090"

$`R-MMU-5638016`
[1] "ENSMUSG00000025035" "ENSMUSG00000046562" "ENSMUSG00000060090"

$`R-MMU-141186`
[1] "ENSMUSG00000025037"

$`R-MMU-374909`
[1] "ENSMUSG00000025037"

$`R-MMU-379382`
[1] "ENSMUSG00000025037"

[1] "ENSMUSG00000025358" "ENSMUSG00000027330" "ENSMUSG00000027715"
[4] "ENSMUSG00000027793" "ENSMUSG00000032477"

$`R-MMU-174164`
[1] "ENSMUSG00000025358" "ENSMUSG00000027715" "ENSMUSG00000027793"
[4] "ENSMUSG00000031016" "ENSMUSG00000040957"

$`R-MMU-174273`
[1] "ENSMUSG00000025358" "ENSMUSG00000027715" "ENSMUSG00000027793"

$`R-MMU-6805109`
[1] "ENSMUSG00000025358" "ENSMUSG00000027715" "ENSMUSG00000027793"
[4] "ENSMUSG00000059552"

$`R-MMU-9624120`
[1] "ENSMUSG00000025358" "ENSMUSG00000034957"

$`R-MMU-9686521`
[1] "ENSMUSG00000025358" "ENSMUSG00000025925" "ENSMUSG00000027715"

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [9]:
%%R
rxns <- rxn2ensembls.nls %>% names()
print(length(rxns))
print(rxns)

 [501] "R-MMU-5626938" "R-MMU-8985343" "R-MMU-9690401" "R-MMU-9690410"
 [505] "R-MMU-9690416" "R-MMU-9690868" "R-MMU-9690898" "R-MMU-9692133"
 [509] "R-MMU-9692136" "R-MMU-9692137" "R-MMU-9692138" "R-MMU-9692139"
 [513] "R-MMU-9692145" "R-MMU-9692156" "R-MMU-9692157" "R-MMU-9692268"
 [517] "R-MMU-450434"  "R-MMU-450466"  "R-MMU-450551"  "R-MMU-450580" 
 [521] "R-MMU-72103"   "R-MMU-9770119" "R-MMU-9770141" "R-MMU-9770142"
 [525] "R-MMU-9770145" "R-MMU-9770236" "R-MMU-6798748" "R-MMU-6800434"
 [529] "R-MMU-6791218" "R-MMU-6791222" "R-MMU-1605595" "R-MMU-1605624"
 [533] "R-MMU-1605717" "R-MMU-6798751" "R-MMU-9840795" "R-MMU-9840833"
 [537] "R-MMU-9840884" "R-MMU-2730692" "R-MMU-373075"  "R-MMU-9696268"
 [541] "R-MMU-70420"   "R-MMU-71218"   "R-MMU-8940388" "R-MMU-2002466"
 [545] "R-MMU-2022141" "R-MMU-2129375" "R-MMU-2395340" "R-MMU-8851044"
 [549] "R-MMU-8938431" "R-MMU-1912394" "R-MMU-2032794" "R-MMU-2065441"
 [553] "R-MMU-212356"  "R-MMU-2247925" "R-MMU-4396387" "R-MMU-5211235"
 [557]

 [961] "R-MMU-2187309" "R-MMU-6810233" "R-MMU-6810234" "R-MMU-6810235"
 [965] "R-MMU-6810238" "R-MMU-6814549" "R-MMU-6814555" "R-MMU-6814559"
 [969] "R-MMU-6814885" "R-MMU-9023840" "R-MMU-9761027" "R-MMU-199032" 
 [973] "R-MMU-265422"  "R-MMU-376419"  "R-MMU-382096"  "R-MMU-5334854"
 [977] "R-MMU-5617960" "R-MMU-5617972" "R-MMU-5617990" "R-MMU-5617999"
 [981] "R-MMU-5634103" "R-MMU-5693628" "R-MMU-9009817" "R-MMU-3928578"
 [985] "R-MMU-3928591" "R-MMU-3928592" "R-MMU-3928595" "R-MMU-3928597"
 [989] "R-MMU-3928598" "R-MMU-3928599" "R-MMU-3928601" "R-MMU-3928604"
 [993] "R-MMU-3928607" "R-MMU-3928612" "R-MMU-3928613" "R-MMU-3928614"
 [997] "R-MMU-3928615" "R-MMU-3928617" "R-MMU-3928624" "R-MMU-3928625"
[1001] "R-MMU-3928627" "R-MMU-3928628" "R-MMU-3928630" "R-MMU-3928631"
[1005] "R-MMU-3928633" "R-MMU-3928635" "R-MMU-3928639" "R-MMU-3928641"
[1009] "R-MMU-3928645" "R-MMU-3928657" "R-MMU-4093329" "R-MMU-4093331"
[1013] "R-MMU-4093332" "R-MMU-4093336" "R-MMU-4093339" "R-MMU-9696266"
[1017]

[1421] "R-MMU-442291"  "R-MMU-442314"  "R-MMU-445064"  "R-MMU-445072" 
[1425] "R-MMU-451347"  "R-MMU-451366"  "R-MMU-5218827" "R-MMU-5218839"
[1429] "R-MMU-5218841" "R-MMU-5218850" "R-MMU-5357445" "R-MMU-5357472"
[1433] "R-MMU-5357483" "R-MMU-5623622" "R-MMU-5623632" "R-MMU-5623667"
[1437] "R-MMU-5625893" "R-MMU-5625959" "R-MMU-5626469" "R-MMU-5627775"
[1441] "R-MMU-5663241" "R-MMU-5665656" "R-MMU-5665802" "R-MMU-5665809"
[1445] "R-MMU-5668714" "R-MMU-5668718" "R-MMU-5668731" "R-MMU-5668735"
[1449] "R-MMU-5671970" "R-MMU-5671993" "R-MMU-5672083" "R-MMU-5672304"
[1453] "R-MMU-8848618" "R-MMU-8875591" "R-MMU-8951586" "R-MMU-8981447"
[1457] "R-MMU-8981926" "R-MMU-8981931" "R-MMU-9013143" "R-MMU-9013144"
[1461] "R-MMU-9013145" "R-MMU-9013147" "R-MMU-9032798" "R-MMU-9691168"
[1465] "R-MMU-9691174" "R-MMU-9751201" "R-NUL-9033341" "R-MMU-111439" 
[1469] "R-MMU-158441"  "R-MMU-158447"  "R-MMU-158481"  "R-MMU-158484" 
[1473] "R-MMU-159046"  "R-MMU-159050"  "R-MMU-170796"  "R-MMU-191830" 
[1477]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[2313] "R-MMU-5696465" "R-MMU-6785732" "R-MMU-6785986" "R-MMU-193068" 
[2317] "R-MMU-193070"  "R-MMU-193072"  "R-MMU-193099"  "R-MMU-5696213"
[2321] "R-MMU-5696220" "R-MMU-6794347" "R-MMU-6794355" "R-MMU-114256" 
[2325] "R-MMU-6805507" "R-MMU-904830"  "R-MMU-917891"  "R-MMU-390281" 
[2329] "R-MMU-9033235" "R-MMU-9033236" "R-MMU-199895"  "R-MMU-442724" 
[2333] "R-MMU-9029434" "R-MMU-2160851" "R-MMU-964737"  "R-MMU-964825" 
[2337] "R-MMU-964830"  "R-MMU-156723"  "R-MMU-984648"  "R-MMU-71046"  
[2341] "R-MMU-5688786" "R-MMU-5691004" "R-MMU-3221864" "R-MMU-8863858"
[2345] "R-MMU-8863914" "R-MMU-8951595" "R-MMU-901047"  "R-NUL-1236962"
[2349] "R-MMU-350578"  "R-MMU-210426"  "R-MMU-210430"  "R-MMU-372505" 
[2353] "R-MMU-372529"  "R-MMU-374899"  "R-MMU-374922"  "R-MMU-380574" 
[2357] "R-MMU-380869"  "R-MMU-380901"  "R-MMU-380905"  "R-MMU-388824" 
[2361] "R-MMU-3322001" "R-MMU-3322003" "R-MMU-3322005" "R-MMU-3322025"
[2365] "R-MMU-3322041" "R-MMU-3322057" "R-MMU-3781018" "R-MMU-3781023"
[2369]

...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)




[6753] "R-MMU-174389"  "R-MMU-174392"  "R-MMU-1368145" "R-MMU-5663266"
[6757] "R-MMU-5686430" "R-MMU-5686475" "R-MMU-8854466" "R-MMU-8855111"
[6761] "R-MMU-8855232" "R-MMU-8855259" "R-MMU-193691"  "R-MMU-2730894"
[6765] "R-MMU-448703"  "R-MMU-5660660" "R-MMU-5660666" "R-MMU-8865237"
[6769] "R-MMU-9604480" "R-MMU-1299304" "R-MMU-5250209" "R-MMU-5250210"
[6773] "R-MMU-5250217" "R-MMU-5672027" "R-MMU-3697882" "R-MMU-8940300"
[6777] "R-MMU-1296348" "R-MMU-139905"  "R-NUL-3139045" "R-MMU-1855177"
[6781] "R-MMU-445075"  "R-MMU-1482695" "R-MMU-75886"   "R-MMU-5334050"
[6785] "R-MMU-212007"  "R-MMU-5362525" "R-MMU-2404134" "R-MMU-2404135"
[6789] "R-MMU-2453863" "R-MMU-2453876" "R-MMU-508040"  "R-MMU-2045911"
[6793] "R-MMU-8985822" "R-MMU-8985829" "R-MMU-9010235" "R-MMU-5624130"
[6797] "R-MMU-5624133" "R-MMU-5638004" "R-MMU-5638006" "R-MMU-5638007"
[6801] "R-MMU-5638016" "R-MMU-141186"  "R-MMU-374909"  "R-MMU-379382" 
[6805] "R-MMU-379395"  "R-MMU-9677917" "R-MMU-8948139" "R-MMU-8948143"
[6809

[7213] "R-MMU-444572"  "R-MMU-189062"  "R-MMU-140847"  "R-MMU-9023909"
[7217] "R-MMU-2201341" "R-MMU-8848246" "R-MMU-420202"  "R-MMU-2247510"
[7221] "R-MMU-186511"  "R-MMU-201595"  "R-MMU-8858428" "R-MMU-448708" 
[7225] "R-MMU-8987012" "R-MMU-5660470" "R-MMU-5660533" "R-MMU-8875902"
[7229] "R-MMU-9675215" "R-MMU-9676254" "R-MMU-9676257" "R-MMU-9676264"
[7233] "R-MMU-9676268" "R-MMU-373738"  "R-MMU-443774"  "R-MMU-6786652"
[7237] "R-MMU-6806831" "R-MMU-5617178" "R-MMU-70606"   "R-MMU-8874979"
[7241] "R-MMU-6783939" "R-MMU-451033"  "R-MMU-70982"   "R-MMU-109903" 
[7245] "R-MMU-373339"  "R-MMU-140664"  "R-MMU-140686"  "R-MMU-9015122"
[7249] "R-MMU-376141"  "R-MMU-428888"  "R-MMU-9706287" "R-MMU-3222237"
[7253] "R-MMU-8959798" "R-MMU-176606"  "R-MMU-9635801" "R-MMU-1861621"
[7257] "R-MMU-199161"  "R-MMU-381026"  "R-MMU-381135"  "R-MMU-381186" 
[7261] "R-MMU-420818"  "R-MMU-389639"  "R-MMU-8939880" "R-MMU-8939892"
[7265] "R-MMU-446204"  "R-MMU-2562594" "R-MMU-194632"  "R-MMU-194689" 
[7269]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[9385] "R-MMU-9699581" "R-MMU-5619439" "R-MMU-417896"  "R-MMU-9634937"
[9389] "R-MMU-381608"  "R-MMU-8959510" "R-MMU-9034714" "R-MMU-9615249"
[9393] "R-MMU-3296326" "R-MMU-5205798" "R-MMU-5205820" "R-MMU-5229203"
[9397] "R-MMU-5661126" "R-MMU-5423038" "R-MMU-6805740" "R-MMU-428585" 
[9401] "R-MMU-9844587" "R-MMU-1614544" "R-MMU-742345"  "R-MMU-417820" 
[9405] "R-MMU-9707606" "R-MMU-9707683" "R-MMU-9707856" "R-MMU-1483087"
[9409] "R-MMU-1483211" "R-MMU-1483219" "R-MMU-1483229" "R-MMU-5693691"
[9413] "R-MMU-9749647" "R-MMU-9749792" "R-MMU-5696119" "R-MMU-9673827"
[9417] "R-MMU-2872444" "R-MMU-204617"  "R-MMU-204647"  "R-MMU-204662" 
[9421] "R-MMU-1482932" "R-MMU-451360"  "R-MMU-5682285" "R-MMU-517536" 
[9425] "R-MMU-191983"  "R-MMU-2161660" "R-MMU-727740"  "R-MMU-727768" 
[9429] "R-MMU-1483096" "R-MMU-1483159" "R-MMU-417842"  "R-MMU-429786" 
[9433] "R-MMU-190782"  "R-MMU-191072"  "R-MMU-375340"  "R-MMU-174959" 
[9437] "R-MMU-174963"  "R-MMU-8851679" "R-MMU-444171"  "R-MMU-9831007"
[9441]

In [10]:
%%R
rxn_knn_misclass_rate.nls <- list()
rxn_knn_ari.nls <- list()
rxn_knn_ecount.nls <- list()
rxn_pca.nls <- list()
count <- 0

In [11]:
%%R
toi_indices <- seq(1,length(tcdd_dose_detail.vec))
print(toi_indices)

 [1]  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25
[26] 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50
[51] 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75
[76] 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94


In [12]:
%%R
tcdd_dose_detail_vec_tis_of_interest <-
   tcdd_dose_detail.vec[toi_indices]
print(tcdd_dose_detail_vec_tis_of_interest)

 [1]  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
[26]  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 30 30 30
[51] 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30
[76] 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30


In [13]:
%%R
vst.count.mtx.tis_of_interest <- vst.count.mtx[, toi_indices]
print(vst.count.mtx.tis_of_interest)

                   SRR7817613 SRR7817695 SRR7817698 SRR1636593 SRR7817637
ENSMUSG00000000001  15.657278  15.830531  15.521473  16.331873  15.737249
ENSMUSG00000000028   7.896070   7.084628   7.449454   8.330852   7.604541
ENSMUSG00000000049  20.359191  19.954578  20.186495  20.383123  20.399895
ENSMUSG00000000058   8.448876   8.629453   8.447006   8.852873   9.186141
ENSMUSG00000000085  10.697905  10.738284  10.429383  10.398967  11.373079
ENSMUSG00000000088  16.775240  16.685460  16.624890  16.029662  16.371583
ENSMUSG00000000120   9.944731   8.569186   9.268519   7.844952   8.894102
ENSMUSG00000000126   7.184312   7.756609   7.866856   6.874071   6.925158
ENSMUSG00000000127  10.624489   9.557760   9.824037  10.662614  10.364958
ENSMUSG00000000142  10.038872   9.304681  10.284228  10.020320  10.620927
ENSMUSG00000000149  16.013477  15.956277  15.867528  14.996136  16.393286
ENSMUSG00000000154  15.658438  15.625613  15.695522  15.342378  15.744391
ENSMUSG00000000168  15.080008  14.3964

ENSMUSG00000001105  13.685235  13.904600  13.670822  13.076574  13.013098
ENSMUSG00000001119   9.566425   9.631210   9.990611   9.767335   9.433160
ENSMUSG00000001123  18.554219  17.849894  18.159200  18.388696  18.385946
ENSMUSG00000001127  15.209370  15.101739  15.429678  14.815247  15.488661
ENSMUSG00000001128  11.856397  11.079074  11.842890  10.886366  10.649347
ENSMUSG00000001131   5.772151   5.823939   6.556267   6.608881   6.710842
ENSMUSG00000001143  11.937988  11.845639  11.682562  11.114515  11.338919
ENSMUSG00000001155  18.899980  18.657120  18.668680  18.089899  18.910979
ENSMUSG00000001158  11.427522  10.366134  10.530753  11.561313  11.384711
ENSMUSG00000001173  10.531956  10.185303  10.302424  11.108415  10.444839
ENSMUSG00000001175  17.380807  17.519977  17.435217  16.736932  17.222410
ENSMUSG00000001211  17.078372  16.877775  16.978084  16.273052  17.042583
ENSMUSG00000001225   6.841439   5.823939   5.801464   5.817700   5.788402
ENSMUSG00000001228   7.353223   7.8025

ENSMUSG00000002249  11.036856  10.868493  11.382105  10.192282  11.286161
ENSMUSG00000002250  13.972019  13.320903  14.684736  10.119642   9.749703
ENSMUSG00000002257   7.413843   6.629107   6.791733   5.817700   5.788402
ENSMUSG00000002279  13.655608  13.968423  13.882522  12.935723  13.803005
ENSMUSG00000002289  14.051196  14.939343  14.844777  13.937317  14.086655
ENSMUSG00000002297   8.918785   8.565784   9.098950   8.196921   8.520145
ENSMUSG00000002307  11.453471  11.261543  11.358450  10.743107  11.577778
ENSMUSG00000002308   9.444808   8.729196   9.803239   9.352800   8.943011
ENSMUSG00000002324   8.636731   8.498791   8.242605   7.747335   8.438533
ENSMUSG00000002325  13.747645  12.921939  13.584650  13.007717  13.561626
ENSMUSG00000002326  11.841327  11.596322  11.874952  11.940662  11.455192
ENSMUSG00000002346  15.926832  15.879926  16.106943  15.172361  15.645202
ENSMUSG00000002365  14.046517  13.969606  13.949512  14.860196  14.311090
ENSMUSG00000002379  14.216566  14.2792

ENSMUSG00000003379   6.978096   7.248006   7.533337   7.214822   7.362724
ENSMUSG00000003402  16.118127  16.163072  16.210396  15.747523  15.910229
ENSMUSG00000003411   7.184312   7.248006   7.356780   6.874071   7.377295
ENSMUSG00000003418   5.772151   7.248006   5.801464   6.874071   6.513963
ENSMUSG00000003421  12.772475  12.502575  12.422478  11.569327  12.859808
ENSMUSG00000003429  14.502483  14.079656  14.790110  14.905432  13.236353
ENSMUSG00000003435  14.810739  14.727862  15.282298  14.212871  14.852820
ENSMUSG00000003436   5.772151   5.823939   5.801464   5.817700   5.788402
ENSMUSG00000003437  13.191945  13.128634  13.068612  13.288265  13.373915
ENSMUSG00000003444   9.397773   8.958334   9.752947   8.847271   8.632144
ENSMUSG00000003452   5.772151   7.088062   5.801464   6.874071   7.061597
ENSMUSG00000003458  14.160890  13.757168  13.848929  14.281083  14.436014
ENSMUSG00000003464  15.836717  15.536413  15.855920  16.094661  15.918842
ENSMUSG00000003476   7.087415   6.8994

ENSMUSG00000015149  14.306637  14.280944  14.226837  14.676133  14.548962
ENSMUSG00000015165  15.121948  16.439584  15.201551  15.063373  14.626021
ENSMUSG00000015214  13.181755  12.927184  13.725936  14.233846  13.756440
ENSMUSG00000015243  17.096526  16.519394  16.726345  16.895671  17.321559
ENSMUSG00000015291  15.568847  15.069524  14.790466  15.086376  15.307302
ENSMUSG00000015314   5.817056   6.882780   6.944110   6.580462   7.089285
ENSMUSG00000015340   9.661471  10.872742  12.127065  10.824891  11.708409
ENSMUSG00000015341  13.705479  13.710894  13.594466  13.618343  13.656377
ENSMUSG00000015342   9.910001  10.129071  10.275186   9.947466  10.263876
ENSMUSG00000015354   7.710323   7.901375   8.422260   7.168522   8.340786
ENSMUSG00000015355   7.751413   8.488315   8.814040   7.527968   8.525631
ENSMUSG00000015357  16.300286  18.164022  17.193139  14.474010  15.666110
ENSMUSG00000015377  11.069021  11.476992  10.319349  10.543975  10.758227
ENSMUSG00000015405   7.211408   6.6817

...

ENSMUSG00000003037  14.038912  14.096687  14.323320  14.131142  14.168999
ENSMUSG00000003039  14.337052  14.418670  13.478381  14.173663  14.382562
ENSMUSG00000003051   8.010148   9.642620   7.420894   8.270682   9.403190
ENSMUSG00000003053  14.242190  12.140176  18.613810  16.554011  19.143158
ENSMUSG00000003062  10.639731  11.353339  10.324592  11.063912  10.377158
ENSMUSG00000003068  14.435096  14.182111  14.487136  14.938886  15.240797
ENSMUSG00000003070   7.051369   6.377995   5.762125   6.406883   7.116627
ENSMUSG00000003072  16.556013  17.013521  16.842349  16.680320  16.459336
ENSMUSG00000003099  13.379907  13.395051  13.970485  13.188901  12.706649
ENSMUSG00000003119  12.932138  12.052854  12.104935  12.635320  12.687205
ENSMUSG00000003131  14.291096  14.188337  13.841799  13.690595  13.395616
ENSMUSG00000003135  11.670431  11.705808  11.744481  12.025603  12.023930
ENSMUSG00000003153   7.750646   8.272645   7.902740   7.748154   8.333982
ENSMUSG00000003161  12.602337  13.4056

ENSMUSG00000004263  12.150845  12.576628  11.880478  12.228584  12.733303
ENSMUSG00000004264  16.009695  16.297565  16.352974  16.375828  15.940214
ENSMUSG00000004266  12.706113  13.104283  13.050680  13.160567  12.677490
ENSMUSG00000004267   6.950512   7.470611   7.124380   7.545464   6.841712
ENSMUSG00000004268  12.745833  12.980303  13.245190  13.797231  13.304273
ENSMUSG00000004270  13.430339  13.671671  14.166764  13.091368  14.475893
ENSMUSG00000004285  12.897355  13.442501  13.014103  12.707613  12.866018
ENSMUSG00000004296   6.443944   6.377995   6.649229   6.406883   6.441083
ENSMUSG00000004317  10.986156  11.679212  11.965254  10.892016  11.221310
ENSMUSG00000004319  15.013673  14.424444  14.977849  14.863808  15.196596
ENSMUSG00000004328   7.316718   7.712176   6.803681   7.083675   6.710895
ENSMUSG00000004341   5.766752   6.377995   5.762125   6.621463   6.995241
ENSMUSG00000004344   5.766752   5.746283   5.762125   5.755262   5.777381
ENSMUSG00000004347   7.316718   6.9444

ENSMUSG00000005371  12.969718  13.075033  12.840130  12.997100  12.606347
ENSMUSG00000005373  11.499210  12.387887  15.966133  14.735736  16.189791
ENSMUSG00000005378  13.020253  12.866716  12.846282  13.106041  12.231973
ENSMUSG00000005397  14.847745  15.571785  16.070228  15.349247  15.921733
ENSMUSG00000005410   9.259205  10.477538  11.723402  10.724893   9.563439
ENSMUSG00000005413  15.954097  17.522779  15.890327  16.517185  15.279120
ENSMUSG00000005447   8.220942   8.903429   8.834404   8.668658   8.640782
ENSMUSG00000005465   9.156073   8.894826   8.747490   9.159553   8.602459
ENSMUSG00000005469  12.434390  12.707301  13.297199  12.476454  13.409217
ENSMUSG00000005474   5.766752   5.746283   5.762125   5.755262   5.777381
ENSMUSG00000005481  12.400031  13.213091  12.531269  12.644807  11.710066
ENSMUSG00000005483  11.777344  10.925175  11.759337  11.725336  11.456316
ENSMUSG00000005493   6.443944   6.377995   5.762125   6.621463   6.478301
ENSMUSG00000005510  13.724374  14.4192

ENSMUSG00000006538   7.713178   7.792982   6.926924   7.164918   7.951600
ENSMUSG00000006542   6.517771   7.192423   6.764423   5.755262   5.777381
ENSMUSG00000006567  11.523403  11.151831  12.161147  11.199377  12.991697
ENSMUSG00000006574   5.766752   6.730599   6.429010   6.406883   7.875675
ENSMUSG00000006576   7.232806   8.437624   7.705153   7.689280   7.986140
ENSMUSG00000006585   9.298863  10.338076  10.530548  10.235863   9.795017
ENSMUSG00000006589  13.804219  14.087641  13.895933  13.877284  13.608066
ENSMUSG00000006599  14.530104  13.734252  13.918222  14.096313  13.357499
ENSMUSG00000006611  14.339647  14.303309  14.831613  14.761066  14.010064
ENSMUSG00000006641   9.659459   9.473023   9.733655   9.945806  10.108551
ENSMUSG00000006649   6.443944   7.107104   6.666735   5.755262   6.707126
ENSMUSG00000006676  15.531259  15.131797  15.283421  15.444899  15.223732
ENSMUSG00000006678   9.141939   9.209265   9.846160   9.464965   9.387907
ENSMUSG00000006699  16.637318  16.5626

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [14]:
%%R
training_indices <-
   caret::createDataPartition(
      tcdd_dose_detail_vec_tis_of_interest,
      times = 1,
      p = 1.0, # no data will be held out when set to "1.0"
      list = FALSE
   )
print(training_indices)

In [15]:
%%R
vst.count.mtx.train <-
   vst.count.mtx.tis_of_interest[, training_indices] #9/10ths of data
vst.count.mtx.test  <-
   vst.count.mtx.tis_of_interest[, -training_indices] #1/10th of data

print(vst.count.mtx.train)

                   SRR7817613 SRR7817695 SRR7817698 SRR1636593 SRR7817637
ENSMUSG00000000001  15.657278  15.830531  15.521473  16.331873  15.737249
ENSMUSG00000000028   7.896070   7.084628   7.449454   8.330852   7.604541
ENSMUSG00000000049  20.359191  19.954578  20.186495  20.383123  20.399895
ENSMUSG00000000058   8.448876   8.629453   8.447006   8.852873   9.186141
ENSMUSG00000000085  10.697905  10.738284  10.429383  10.398967  11.373079
ENSMUSG00000000088  16.775240  16.685460  16.624890  16.029662  16.371583
ENSMUSG00000000120   9.944731   8.569186   9.268519   7.844952   8.894102
ENSMUSG00000000126   7.184312   7.756609   7.866856   6.874071   6.925158
ENSMUSG00000000127  10.624489   9.557760   9.824037  10.662614  10.364958
ENSMUSG00000000142  10.038872   9.304681  10.284228  10.020320  10.620927
ENSMUSG00000000149  16.013477  15.956277  15.867528  14.996136  16.393286
ENSMUSG00000000154  15.658438  15.625613  15.695522  15.342378  15.744391
ENSMUSG00000000168  15.080008  14.3964

ENSMUSG00000001105  13.685235  13.904600  13.670822  13.076574  13.013098
ENSMUSG00000001119   9.566425   9.631210   9.990611   9.767335   9.433160
ENSMUSG00000001123  18.554219  17.849894  18.159200  18.388696  18.385946
ENSMUSG00000001127  15.209370  15.101739  15.429678  14.815247  15.488661
ENSMUSG00000001128  11.856397  11.079074  11.842890  10.886366  10.649347
ENSMUSG00000001131   5.772151   5.823939   6.556267   6.608881   6.710842
ENSMUSG00000001143  11.937988  11.845639  11.682562  11.114515  11.338919
ENSMUSG00000001155  18.899980  18.657120  18.668680  18.089899  18.910979
ENSMUSG00000001158  11.427522  10.366134  10.530753  11.561313  11.384711
ENSMUSG00000001173  10.531956  10.185303  10.302424  11.108415  10.444839
ENSMUSG00000001175  17.380807  17.519977  17.435217  16.736932  17.222410
ENSMUSG00000001211  17.078372  16.877775  16.978084  16.273052  17.042583
ENSMUSG00000001225   6.841439   5.823939   5.801464   5.817700   5.788402
ENSMUSG00000001228   7.353223   7.8025

ENSMUSG00000002249  11.036856  10.868493  11.382105  10.192282  11.286161
ENSMUSG00000002250  13.972019  13.320903  14.684736  10.119642   9.749703
ENSMUSG00000002257   7.413843   6.629107   6.791733   5.817700   5.788402
ENSMUSG00000002279  13.655608  13.968423  13.882522  12.935723  13.803005
ENSMUSG00000002289  14.051196  14.939343  14.844777  13.937317  14.086655
ENSMUSG00000002297   8.918785   8.565784   9.098950   8.196921   8.520145
ENSMUSG00000002307  11.453471  11.261543  11.358450  10.743107  11.577778
ENSMUSG00000002308   9.444808   8.729196   9.803239   9.352800   8.943011
ENSMUSG00000002324   8.636731   8.498791   8.242605   7.747335   8.438533
ENSMUSG00000002325  13.747645  12.921939  13.584650  13.007717  13.561626
ENSMUSG00000002326  11.841327  11.596322  11.874952  11.940662  11.455192
ENSMUSG00000002346  15.926832  15.879926  16.106943  15.172361  15.645202
ENSMUSG00000002365  14.046517  13.969606  13.949512  14.860196  14.311090
ENSMUSG00000002379  14.216566  14.2792

ENSMUSG00000003379   6.978096   7.248006   7.533337   7.214822   7.362724
ENSMUSG00000003402  16.118127  16.163072  16.210396  15.747523  15.910229
ENSMUSG00000003411   7.184312   7.248006   7.356780   6.874071   7.377295
ENSMUSG00000003418   5.772151   7.248006   5.801464   6.874071   6.513963
ENSMUSG00000003421  12.772475  12.502575  12.422478  11.569327  12.859808
ENSMUSG00000003429  14.502483  14.079656  14.790110  14.905432  13.236353
ENSMUSG00000003435  14.810739  14.727862  15.282298  14.212871  14.852820
ENSMUSG00000003436   5.772151   5.823939   5.801464   5.817700   5.788402
ENSMUSG00000003437  13.191945  13.128634  13.068612  13.288265  13.373915
ENSMUSG00000003444   9.397773   8.958334   9.752947   8.847271   8.632144
ENSMUSG00000003452   5.772151   7.088062   5.801464   6.874071   7.061597
ENSMUSG00000003458  14.160890  13.757168  13.848929  14.281083  14.436014
ENSMUSG00000003464  15.836717  15.536413  15.855920  16.094661  15.918842
ENSMUSG00000003476   7.087415   6.8994

ENSMUSG00000004654   5.772151   5.823939   5.801464   6.874071   5.788402
ENSMUSG00000004655  14.747756  13.087596  14.463217  14.827505  14.821809
ENSMUSG00000004665   9.278459   8.779877   9.139352   9.162351   8.820055
ENSMUSG00000004667  12.630759  12.664845  12.829409  11.918410  13.129548
ENSMUSG00000004668   5.772151   5.823939   5.801464   5.817700   5.788402
ENSMUSG00000004677  13.315481  13.379315  13.441685  12.216361  14.002116
ENSMUSG00000004709   6.461389   6.629107   7.129405   6.608881   5.788402
ENSMUSG00000004730  10.711675  10.319755  10.487779  10.736089  10.181033
ENSMUSG00000004768  10.858896  10.186447   9.984629  12.005461  11.139118
ENSMUSG00000004771  14.709691  14.481897  14.309815  14.484534  14.810737
ENSMUSG00000004788  12.098791  12.066343  11.844121  10.999319  11.995076
ENSMUSG00000004789  17.191531  16.987719  17.208101  17.205223  17.013286
ENSMUSG00000004791   5.772151   5.823939   5.801464   6.608881   5.788402
ENSMUSG00000004815  14.053769  13.9102

...

ENSMUSG00000003099  13.535561  12.624672  13.422628  13.616044
ENSMUSG00000003119  11.590418  12.737374  12.323477  12.607004
ENSMUSG00000003131  13.788311  14.280723  13.205494  14.331272
ENSMUSG00000003135  11.594613  11.296960  11.825780  11.525318
ENSMUSG00000003153   8.217791   8.169814   7.704427   8.340363
ENSMUSG00000003161  13.007364  12.778032  13.510510  12.820689
ENSMUSG00000003184  14.167557  13.245122  13.748516  13.572289
ENSMUSG00000003200  12.742886  11.644278  11.823612  12.521644
ENSMUSG00000003206   8.642864   7.784022   7.175802   8.121298
ENSMUSG00000003208   9.193910   9.009202   9.067916   9.257044
ENSMUSG00000003226  15.561604  16.739406  16.204343  16.150858
ENSMUSG00000003227   5.776276   6.573331   6.088529   5.742467
ENSMUSG00000003228   9.358271  10.577566   9.407051  10.444315
ENSMUSG00000003233  12.544568  12.421407  11.648913  12.611183
ENSMUSG00000003235  14.003057  13.135340  13.271059  13.273882
ENSMUSG00000003269  11.547456  10.906705  11.690075  11

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



   5.712309   5.745631   5.742467
ENSMUSG00000010095  17.095878  17.625826  14.133409  18.119272
ENSMUSG00000010097  15.016018  14.576670  14.739682  14.596754
ENSMUSG00000010110  14.479377  14.003705  14.474091  13.976186
ENSMUSG00000010122  11.622334   9.374640  16.728297   9.128158
ENSMUSG00000010142   8.701955   8.370161   7.101547   8.566836
ENSMUSG00000010311   6.474725   5.712309   5.745631   6.569632
ENSMUSG00000010376  14.734568  13.851564  14.068669  14.188654
ENSMUSG00000010392  13.411987  13.700335  13.981839  13.656777
ENSMUSG00000010406  12.920679  11.533711  11.658759  12.088531
ENSMUSG00000010453  14.915710  15.007310  14.482362  14.674773
ENSMUSG00000010461   7.379654   7.466836   7.471586   7.494942
ENSMUSG00000010601  12.806918   9.057086  16.158629   9.569295
ENSMUSG00000010607  10.778485  10.419656  11.343330  10.963316
ENSMUSG00000010608  15.186707  15.829248  16.306357  15.041229
ENSMUSG00000010651  18.447315  17.624405  17.128430  17.693800
ENSMUSG00000010660   

ENSMUSG00000015165  14.880713  14.912904  14.873216  14.995359
ENSMUSG00000015214  13.162496  13.456985  13.605958  13.307016
ENSMUSG00000015243  15.969565  16.788560  16.944524  16.312211
ENSMUSG00000015291  14.928379  14.063579  14.730323  14.930878
ENSMUSG00000015314   7.937236   7.629192   7.104519   7.934686
ENSMUSG00000015340  13.634824  14.241499  11.046350  14.153404
ENSMUSG00000015341  14.219752  13.989469  14.166799  14.367007
ENSMUSG00000015342   9.910285   9.336106   9.346296   9.462616
ENSMUSG00000015354   7.472646   7.826474   7.357224   8.213419
ENSMUSG00000015355  11.143686  10.208171   8.435690  10.205044
ENSMUSG00000015357  13.892009  14.346829  14.558375  15.180387
ENSMUSG00000015377  10.725666  10.285566  10.935567  10.345044
ENSMUSG00000015405   5.776276   6.826717   8.101231   7.075983
ENSMUSG00000015437   6.706428   6.573331   7.674078   7.154959
ENSMUSG00000015452   7.554598   7.192633   6.933533   7.938177
ENSMUSG00000015461  12.837450  12.560972  11.683212  13

ENSMUSG00000017376  10.706063  11.439300  11.228425  11.224629
ENSMUSG00000017390   8.673295   7.494926  11.344635   7.781371
ENSMUSG00000017404  15.106048  14.650824  14.453079  14.388743
ENSMUSG00000017412   8.178713   8.004197  10.025471   7.493964
ENSMUSG00000017428  15.361753  15.642380  14.771295  15.476759
ENSMUSG00000017453  13.509818  10.390148  17.244607   9.953098
ENSMUSG00000017466  12.903076  12.745321  10.808592  13.251929
ENSMUSG00000017485  13.557374  14.132931  14.985903  13.763657
ENSMUSG00000017491   9.913061   9.144969   9.704445   9.915918
ENSMUSG00000017493  18.399655  17.186741  19.458458  18.313674
ENSMUSG00000017499   7.791677   7.598045   7.150770   7.148183
ENSMUSG00000017548  12.730868  13.082557  13.436564  12.509591
ENSMUSG00000017588   7.454948   6.673717   7.026905   6.712985
ENSMUSG00000017607   7.597526   7.458000   7.242106   7.589746
ENSMUSG00000017615  14.132863  14.761106  13.313147  15.041745
ENSMUSG00000017631  13.104475  13.161106   9.286851  13

In [16]:
%%R
tcdd_dose_detail.vec.train <-
   tcdd_dose_detail_vec_tis_of_interest[training_indices]
tcdd_dose_detail.vec.test <-
   tcdd_dose_detail_vec_tis_of_interest[-training_indices]

# print(gtex_tissue_detail.vec.train)
print(length(unique(tcdd_dose_detail.vec.train)))

[1] 2


In [17]:
%%R
saveRDS(tcdd_dose_detail.vec.train,file=paste(OUT_DIR,"tcdd_dose_detail_vec_train030_time_cross_70.Rds",sep=""))

In [18]:
%%R
cv_fold_indices <- caret::createFolds(tcdd_dose_detail.vec.train,
                                      k = N_FOLDS)
print(cv_fold_indices)

$Fold01
[1] 18 21 44 45 50 61 68 70 84

$Fold02
[1]  1 15 27 43 55 69 72 75 91

$Fold03
[1]  6  7 10 19 22 49 51 60 79

$Fold04
 [1]  3  4 24 35 39 64 67 73 78 93

$Fold05
 [1]  8 25 30 38 47 52 56 63 74 89

$Fold06
[1] 20 28 33 40 46 48 58 71 80

$Fold07
 [1] 11 13 16 32 36 59 65 76 86 92

$Fold08
[1] 14 23 34 42 62 77 82 87 88

$Fold09
[1]  5 12 17 29 41 53 81 83 90

$Fold10
 [1]  2  9 26 31 37 54 57 66 85 94



In [19]:
%%R
binary_tcdd_dose_annotations <- unique(tcdd_dose_detail.vec)
print(binary_tcdd_dose_annotations)

[1]  0 30


In [20]:
%%R
full_rxn_pca_results.nls <- list()
rxn_id_2_result_file_idx.nls <- list()

In [21]:
%%R
n_rxns <- length(rxns)
print(n_rxns)
result_idx <- 1
for(rxn_id_idx in seq(1:n_rxns)){
   rxn_id <- rxns[rxn_id_idx]
   rxn_pca <-
      prcomp(t(vst.count.mtx.train[rxn2ensembls.nls[[rxn_id]], ]), scale. = T)
   full_rxn_pca_results.nls[[rxn_id]] <- rxn_pca
   rxn_id_2_result_file_idx.nls[[rxn_id]] <- result_idx
   rxn_pca.nls[[rxn_id]] <-
      rxn_pca$x[, 1] # 1st principal component coordinate within this reaction-space for each sample
   if(mod(rxn_id_idx,100) == 0){
      print(paste("Processed ",rxn_id_idx,
                  " of ",n_rxns,
                  " reactions (",round((rxn_id_idx + 1)/n_rxns,digits = 3) * 100,"%)...",
                  sep=""))
      flush.console()
   }
   if(mod(rxn_id_idx,1000) == 0){
      print(paste("Storing PCA objects containing reactions ",rxn_id_idx-1000,
                  "-",rxn_id_idx,
                  " of ",n_rxns,
                  " reactions (",round((rxn_id_idx + 1)/n_rxns,digits = 3) * 100,"%)...",
                  sep=""))
      saveRDS(full_rxn_pca_results.nls,
              paste(OUT_DIR, "full_rxn_pca_results_nls",rxn_id_idx-1000,
                    "-",rxn_id_idx,"030_time_cross_70.Rds", sep=""))
      full_rxn_pca_results.nls <- list()
      gc()
      result_idx <- result_idx + 1
   }
}

[1] 9825
[1] "Processed 100 of 9825 reactions (1%)..."
[1] "Processed 200 of 9825 reactions (2%)..."
[1] "Processed 300 of 9825 reactions (3.1%)..."
[1] "Processed 400 of 9825 reactions (4.1%)..."
[1] "Processed 500 of 9825 reactions (5.1%)..."
[1] "Processed 600 of 9825 reactions (6.1%)..."
[1] "Processed 700 of 9825 reactions (7.1%)..."
[1] "Processed 800 of 9825 reactions (8.2%)..."
[1] "Processed 900 of 9825 reactions (9.2%)..."
[1] "Processed 1000 of 9825 reactions (10.2%)..."
[1] "Storing PCA objects containing reactions 0-1000 of 9825 reactions (10.2%)..."
[1] "Processed 1100 of 9825 reactions (11.2%)..."
[1] "Processed 1200 of 9825 reactions (12.2%)..."
[1] "Processed 1300 of 9825 reactions (13.2%)..."
[1] "Processed 1400 of 9825 reactions (14.3%)..."
[1] "Processed 1500 of 9825 reactions (15.3%)..."
[1] "Processed 1600 of 9825 reactions (16.3%)..."
[1] "Processed 1700 of 9825 reactions (17.3%)..."
[1] "Processed 1800 of 9825 reactions (18.3%)..."
[1] "Processed 1900 of 9825 re

In [22]:
%%R
# store remaining PCA objects and removing from RAM
saveRDS(rxn_id_2_result_file_idx.nls,
        paste(OUT_DIR,"rxn_id_2_result_file_idx_nls030_time_cross_70.Rds",sep=""))
saveRDS(full_rxn_pca_results.nls,
        paste(OUT_DIR, "full_rxn_pca_results_nls030_time_cross_70.Rds", sep=""))
rm(full_rxn_pca_results.nls)
gc()

           used  (Mb) gc trigger  (Mb) max used  (Mb)
Ncells  6951369 371.3   13077600 698.5 11945217 638.0
Vcells 16062943 122.6   25842636 197.2 25842634 197.2


In [23]:
%%R
# compare informaction content of below files with pca plots or similar
saveRDS(rxn_pca.nls, paste(OUT_DIR, "rxn_pca_nls030_time_cross_70.Rds", sep = ""))
saveRDS(vst.count.mtx.train,
        paste(OUT_DIR, "vst_count_mtx_train030_time_cross_70.Rds", sep = ""))

In [24]:
%%R
for (cv_fold in names(cv_fold_indices)) {
  cur_cv_fold_indices <- cv_fold_indices[[cv_fold]]
  print(cur_cv_fold_indices)
  print("----------")
}

[1] 18 21 44 45 50 61 68 70 84
[1] "----------"
[1]  1 15 27 43 55 69 72 75 91
[1] "----------"
[1]  6  7 10 19 22 49 51 60 79
[1] "----------"
 [1]  3  4 24 35 39 64 67 73 78 93
[1] "----------"
 [1]  8 25 30 38 47 52 56 63 74 89
[1] "----------"
[1] 20 28 33 40 46 48 58 71 80
[1] "----------"
 [1] 11 13 16 32 36 59 65 76 86 92
[1] "----------"
[1] 14 23 34 42 62 77 82 87 88
[1] "----------"
[1]  5 12 17 29 41 53 81 83 90
[1] "----------"
 [1]  2  9 26 31 37 54 57 66 85 94
[1] "----------"


In [ ]:
%%R
formatted_annotations <- format(binary_tcdd_dose_annotations, nsmall = 2)

formatted_annotations <- trimws(formatted_annotations)

binary_tcdd_dose_annotations <- formatted_annotations

print(binary_tcdd_dose_annotations)
for (annotation in binary_tcdd_dose_annotations) {
    print(annotation)
}

[1] "0.00"  "30.00"
[1] "0.00"
[1] "30.00"


In [ ]:
%%R
# main loop
for (rxn_id_idx in seq(1:length(rxns))) {
   rxn_id <- rxns[rxn_id_idx]
   ensembl_ids <- rxn2ensembls.nls[[rxn_id]]
   
   mean_misclass_rate <- list()
   sum_ari <- 0

   for (cv_fold in names(cv_fold_indices)) {
      cur_cv_fold_indices <- cv_fold_indices[[cv_fold]]
      
      vst.count.mtx.train.cv_train <-
         vst.count.mtx.train[, -cur_cv_fold_indices] # 4/5ths of training features
      vst.count.mtx.train.cv_test <-
         vst.count.mtx.train[, cur_cv_fold_indices] # 1/5th of training features
      
      tcdd_dose_detail.vec.train.cv_train <-
         tcdd_dose_detail.vec.train[-cur_cv_fold_indices] # 4/5ths of training labels
      tcdd_dose_detail.vec.train.cv_test <-
         tcdd_dose_detail.vec.train[cur_cv_fold_indices] # 1/5th of training labels
      
      binary_tcdd_dose_detail_vec.test.cv_test_list <- list()
      for (tissue_annotation in binary_tcdd_dose_annotations) {
         binary_tcdd_dose_detail_vec.test.cv_test_list[[as.character(tissue_annotation)]] <-
            (as.character(tcdd_dose_detail.vec.train.cv_test) == as.character(tissue_annotation))
      }
      
      cv_train.expr_mat <-
         t(vst.count.mtx.train.cv_train[ensembl_ids, ])
      cv_test.expr_mat <-
         t(vst.count.mtx.train.cv_test[ensembl_ids, ])
      
      rxn_knn_calls <- class::knn(train = cv_train.expr_mat,
                                  test = cv_test.expr_mat,
                                  cl = tcdd_dose_detail.vec.train.cv_train)
      
      # calculate & store adjusted rand index
      cur_ari <- pdfCluster::adj.rand.index(rxn_knn_calls,
                                            tcdd_dose_detail.vec.train.cv_test)
      sum_ari <- cur_ari + sum_ari
      
      # for each tissue, calculate misclassification rate
      for (tissue_annotation in binary_tcdd_dose_annotations) {
         cur_rxn_knn_calls <- (rxn_knn_calls == tissue_annotation)
            
         # calculate misclassification rate (https://stat.ethz.ch/pipermail/r-help/2011-September/288885.html)
         tab <- table(cur_rxn_knn_calls,
                      binary_tcdd_dose_detail_vec.test.cv_test_list[[tissue_annotation]])
        # print(tab)
         cur_misclass_rate <- 1 - sum(diag(tab)) / sum(tab)
        # print(paste("Misclass rate = 1 - ",sum(diag(tab))," / ",sum(tab)," = ", cur_misclass_rate,"...",sep=""))
         sum_misclass_rate <- cur_misclass_rate
         if (!is.null(mean_misclass_rate[[tissue_annotation]])) {
           sum_misclass_rate <- sum_misclass_rate + mean_misclass_rate[[tissue_annotation]]
         }
         mean_misclass_rate[[tissue_annotation]] <- sum_misclass_rate
      }
   }
   for(tissue_annotation in binary_tcdd_dose_annotations){
      mean_misclass_rate[[tissue_annotation]] <- (mean_misclass_rate[[tissue_annotation]] / N_FOLDS)
   }
   mean_ari <- sum_ari / N_FOLDS
   ecount <- length(ensembl_ids)
   
   rxn_knn_misclass_rate.nls[[rxn_id]] <- mean_misclass_rate
   assign("mean_misclass_rate",NULL,envir = .GlobalEnv)
   
   rxn_knn_ari.nls[[rxn_id]] <- mean_ari
   rxn_knn_ecount.nls[[rxn_id]] <- ecount
   
   count <- count + 1
   if (mod(count, 10) == 0) {
      print(
         paste(
            "Last RXN_ID = ",
            rxn_id,
            ": Last ARI = ",
            mean_ari,
            ": Last ECOUNT = ",
            ecount,
            ": Last Lung MISCLASS = ",
            rxn_knn_misclass_rate.nls[[rxn_id]][["Lung"]],
            ": Last Uterus MISCLASS = ",
            rxn_knn_misclass_rate.nls[[rxn_id]][["Uterus"]],
            ". Now ",
            round(1.0 - count / length(rxns),3) * 100,
            "% remaining..."
         )
      )
      flush.console()
   }
}

[1] "Last RXN_ID =  R-MMU-749456 : Last ARI =  1 : Last ECOUNT =  178 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  99.9 % remaining..."
[1] "Last RXN_ID =  R-MMU-9021601 : Last ARI =  1 : Last ECOUNT =  24 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  99.8 % remaining..."
[1] "Last RXN_ID =  R-MMU-3108203 : Last ARI =  1 : Last ECOUNT =  15 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  99.7 % remaining..."
[1] "Last RXN_ID =  R-MMU-9838627 : Last ARI =  0.724007048092869 : Last ECOUNT =  4 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  99.6 % remaining..."
[1] "Last RXN_ID =  R-MMU-193679 : Last ARI =  -0.0172491743863927 : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  99.5 % remaining..."
[1] "Last RXN_ID =  R-MMU-194544 : Last ARI =  0.141627768022632 : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  99.4 % remaining..."
[1] "Last RXN_ID =  R-MMU-3247837 : Last ARI =  0.95970

[1] "Last RXN_ID =  R-MMU-450580 : Last ARI =  0.910555555555555 : Last ECOUNT =  12 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  94.7 % remaining..."
[1] "Last RXN_ID =  R-MMU-6791222 : Last ARI =  0.881923714759536 : Last ECOUNT =  61 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  94.6 % remaining..."
[1] "Last RXN_ID =  R-MMU-9696268 : Last ARI =  1 : Last ECOUNT =  41 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  94.5 % remaining..."
[1] "Last RXN_ID =  R-MMU-1912394 : Last ARI =  1 : Last ECOUNT =  9 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  94.4 % remaining..."
[1] "Last RXN_ID =  R-MMU-8936481 : Last ARI =  0.955 : Last ECOUNT =  77 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  94.3 % remaining..."
[1] "Last RXN_ID =  R-MMU-9677051 : Last ARI =  1 : Last ECOUNT =  10 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  94.2 % remaining..."
[1] "Last RXN_ID =  R-MMU-5336403 : Last ARI =  1 : Last ECOUNT = 

[1] "Last RXN_ID =  R-MMU-174119 : Last ARI =  0.955 : Last ECOUNT =  18 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  89.4 % remaining..."
[1] "Last RXN_ID =  R-MMU-174224 : Last ARI =  0.955 : Last ECOUNT =  18 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  89.3 % remaining..."
[1] "Last RXN_ID =  R-MMU-3788708 : Last ARI =  0.955 : Last ECOUNT =  23 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  89.2 % remaining..."
[1] "Last RXN_ID =  R-MMU-927830 : Last ARI =  0.959701492537313 : Last ECOUNT =  11 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  89.1 % remaining..."
[1] "Last RXN_ID =  R-MMU-448955 : Last ARI =  0.955 : Last ECOUNT =  6 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  89 % remaining..."
[1] "Last RXN_ID =  R-MMU-200652 : Last ARI =  -0.0010527462498516 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  88.9 % remaining..."
[1] "Last RXN_ID =  R-MMU-9624798 : Last ARI =  0.261227197

[1] "Last RXN_ID =  R-MMU-9013004 : Last ARI =  0.955 : Last ECOUNT =  16 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  84.1 % remaining..."
[1] "Last RXN_ID =  R-MMU-9818576 : Last ARI =  0.219695761389133 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  84 % remaining..."
[1] "Last RXN_ID =  R-MMU-2029273 : Last ARI =  0.919402985074627 : Last ECOUNT =  71 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  83.9 % remaining..."
[1] "Last RXN_ID =  R-MMU-8850356 : Last ARI =  0.884411764705882 : Last ECOUNT =  11 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  83.8 % remaining..."
[1] "Last RXN_ID =  R-MMU-1363303 : Last ARI =  0.915257048092869 : Last ECOUNT =  9 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  83.7 % remaining..."
[1] "Last RXN_ID =  R-MMU-188386 : Last ARI =  0.80718954248366 : Last ECOUNT =  4 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  83.6 % remaining..."
[1] "Last RXN_ID =  R-M

[1] "Last RXN_ID =  R-MMU-1449633 : Last ARI =  1 : Last ECOUNT =  5 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  78.9 % remaining..."
[1] "Last RXN_ID =  R-MMU-211910 : Last ARI =  1 : Last ECOUNT =  12 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  78.8 % remaining..."
[1] "Last RXN_ID =  R-MMU-3928646 : Last ARI =  0.955 : Last ECOUNT =  12 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  78.7 % remaining..."
[1] "Last RXN_ID =  R-MMU-8848484 : Last ARI =  0.870812603648424 : Last ECOUNT =  5 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  78.6 % remaining..."
[1] "Last RXN_ID =  R-MMU-9022185 : Last ARI =  0.740514096185738 : Last ECOUNT =  3 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  78.5 % remaining..."
[1] "Last RXN_ID =  R-MMU-2032396 : Last ARI =  0.611326699834162 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  78.4 % remaining..."
[1] "Last RXN_ID =  R-MMU-182990 : Last ARI =  1 : La

...

[1] "Last RXN_ID =  R-MMU-114683 : Last ARI =  0.712958540630182 : Last ECOUNT =  3 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  22.8 % remaining..."
[1] "Last RXN_ID =  R-MMU-75090 : Last ARI =  0.207662520729685 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  22.7 % remaining..."
[1] "Last RXN_ID =  R-MMU-77342 : Last ARI =  0.219083806105311 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  22.6 % remaining..."
[1] "Last RXN_ID =  R-MMU-449937 : Last ARI =  0.00144316946956963 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  22.5 % remaining..."
[1] "Last RXN_ID =  R-MMU-9718020 : Last ARI =  0.115375797306875 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  22.4 % remaining..."
[1] "Last RXN_ID =  R-MMU-6806966 : Last ARI =  0.461622090351456 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  22.3 % remaining..."
[1] "Last RXN_

[1] "Last RXN_ID =  R-NUL-2128982 : Last ARI =  -0.0337698720096986 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  17.8 % remaining..."
[1] "Last RXN_ID =  R-MMU-9757010 : Last ARI =  0.914701492537313 : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  17.7 % remaining..."
[1] "Last RXN_ID =  R-MMU-9653592 : Last ARI =  0.0430179683718568 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  17.6 % remaining..."
[1] "Last RXN_ID =  R-MMU-9668464 : Last ARI =  -0.0378226514486391 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  17.5 % remaining..."
[1] "Last RXN_ID =  R-MMU-2454118 : Last ARI =  0.484784899034241 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  17.4 % remaining..."
[1] "Last RXN_ID =  R-MMU-159425 : Last ARI =  0.959701492537313 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  17.3 % remaining..."
[1] "L

[1] "Last RXN_ID =  R-MMU-9012036 : Last ARI =  3.98133864511221e-05 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  12.7 % remaining..."
[1] "Last RXN_ID =  R-MMU-975633 : Last ARI =  0.700323582888769 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  12.6 % remaining..."
[1] "Last RXN_ID =  R-MMU-9027532 : Last ARI =  0.735257048092869 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  12.5 % remaining..."
[1] "Last RXN_ID =  R-MMU-917888 : Last ARI =  0.364572159283902 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  12.4 % remaining..."
[1] "Last RXN_ID =  R-MMU-6800315 : Last ARI =  0.518215588723051 : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  12.3 % remaining..."
[1] "Last RXN_ID =  R-MMU-200424 : Last ARI =  -0.0156807228915663 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  12.2 % remaining..."
[1] "Las

[1] "Last RXN_ID =  R-MMU-192312 : Last ARI =  0.959701492537313 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  7.6 % remaining..."
[1] "Last RXN_ID =  R-MMU-8879117 : Last ARI =  0.433446590576529 : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  7.5 % remaining..."
[1] "Last RXN_ID =  R-MMU-139909 : Last ARI =  0.0200852311793266 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  7.4 % remaining..."
[1] "Last RXN_ID =  R-MMU-8851979 : Last ARI =  0.204383304361725 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  7.3 % remaining..."
[1] "Last RXN_ID =  R-MMU-9704248 : Last ARI =  0.595514096185738 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  7.2 % remaining..."
[1] "Last RXN_ID =  R-MMU-5629230 : Last ARI =  0.865555555555556 : Last ECOUNT =  4 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  7.1 % remaining..."
[1] "Last RXN_ID 

[1] "Last RXN_ID =  R-MMU-9695949 : Last ARI =  0.492055555555556 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  2.5 % remaining..."
[1] "Last RXN_ID =  R-MMU-1855172 : Last ARI =  0.413553502567484 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  2.4 % remaining..."
[1] "Last RXN_ID =  R-MMU-6807055 : Last ARI =  0.824701492537313 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  2.3 % remaining..."
[1] "Last RXN_ID =  R-MMU-1462039 : Last ARI =  0.616764096185738 : Last ECOUNT =  9 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  2.2 % remaining..."
[1] "Last RXN_ID =  R-MMU-3341294 : Last ARI =  0.28826151107209 : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  2.1 % remaining..."
[1] "Last RXN_ID =  R-MMU-8952128 : Last ARI =  0.796625207296849 : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  2 % remaining..."
[1] "Last RXN_ID = 

In [27]:
%%R
saveRDS(
   rxn_knn_misclass_rate.nls,
   paste(OUT_DIR, "toi_rxn_knn_misclass_rate_nls030_time_cross_70.Rds", sep = "")
)
saveRDS(rxn_knn_ari.nls,
        paste(OUT_DIR, "toi_rxn_knn_ari_nls030_time_cross_70.Rds", sep = ""))
saveRDS(rxn_knn_ecount.nls,
        paste(OUT_DIR, "toi_rxn_knn_ecount_nls030_time_cross_70.Rds", sep = ""))

In [28]:
%%R
d <- data.frame(
   RXN_ID = names(rxn2ensembls.nls),
   MISCLASS = unlist(rxn_knn_misclass_rate.nls),
   ARI = unlist(rxn_knn_ari.nls),
   ECOUNT = unlist(rxn_knn_ecount.nls)
)
print(d)
saveRDS(d, paste(OUT_DIR, "toi_summary_df030_time_cross_70.Rds", sep = ""))

                           RXN_ID MISCLASS           ARI ECOUNT
R-MMU-170666.0.00    R-MMU-170666        0  9.597015e-01     14
R-MMU-170666.30.00   R-MMU-170671        0  9.597015e-01     14
R-MMU-170671.0.00    R-MMU-170674        0  9.597015e-01     14
R-MMU-170671.30.00   R-MMU-170686        0  9.597015e-01     14
R-MMU-170674.0.00    R-MMU-380073        0  1.000000e+00    182
R-MMU-170674.30.00   R-MMU-392187        0  1.000000e+00     22
R-MMU-170686.0.00    R-MMU-392195        0  1.000000e+00     22
R-MMU-170686.30.00   R-MMU-392202        0  1.000000e+00     22
R-MMU-380073.0.00    R-MMU-749454        0  1.000000e+00    178
R-MMU-380073.30.00   R-MMU-749456        0  1.000000e+00    178
R-MMU-392187.0.00    R-MMU-751001        0  1.000000e+00     21
R-MMU-392187.30.00  R-MMU-8964229        0  1.000000e+00      5
R-MMU-392195.0.00   R-MMU-8964242        0  1.000000e+00      5
R-MMU-392195.30.00  R-MMU-8964329        0  1.000000e+00      5
R-MMU-392202.0.00   R-MMU-8982012       

R-MMU-205099.30.00  R-MMU-5654224        0  9.597015e-01     12
R-MMU-205112.0.00   R-MMU-5654392        0  1.000000e+00     20
R-MMU-205112.30.00  R-MMU-5654397        0  9.597015e-01     16
R-MMU-205117.0.00   R-MMU-5654399        0  9.597015e-01     16
R-MMU-205117.30.00  R-MMU-5654402        0  9.597015e-01     20
R-MMU-209532.0.00   R-MMU-5654404        0  9.597015e-01     16
R-MMU-209532.30.00  R-MMU-5654406        0  9.597015e-01     18
R-MMU-209566.0.00   R-MMU-5654407        0  9.597015e-01     16
R-MMU-209566.30.00  R-MMU-5654408        0  9.597015e-01     12
R-MMU-9012761.0.00  R-MMU-5654409        0  9.597015e-01     12
R-MMU-9012761.30.00 R-MMU-5654413        0  1.000000e+00     17
R-MMU-3247837.0.00  R-MMU-5654416        0  9.597015e-01     15
R-MMU-3247837.30.00 R-MMU-5654418        0  1.000000e+00     14
R-MMU-3247840.0.00  R-MMU-5654422        0  1.000000e+00     14
R-MMU-3247840.30.00 R-MMU-5654423        0  1.000000e+00     16
R-MMU-3247843.0.00  R-MMU-5654425       

R-MMU-5654224.30.00 R-MMU-9011082        0  6.439656e-01      2
R-MMU-5654392.0.00  R-MMU-9011144        0  9.113962e-02      3
R-MMU-5654392.30.00 R-MMU-9011146        0  9.113962e-02      3
R-MMU-5654397.0.00  R-MMU-9753028        0  7.512085e-01      5
R-MMU-5654397.30.00 R-MMU-9753126        0  7.512085e-01      5
R-MMU-5654399.0.00   R-MMU-514604        0  4.597968e-01      2
R-MMU-5654399.30.00  R-MMU-975389        0  7.148056e-01      4
R-MMU-5654402.0.00  R-MMU-1169394        0  9.555556e-01      5
R-MMU-5654402.30.00 R-MMU-1169398        0  9.152570e-01      5
R-MMU-5654404.0.00  R-MMU-1169402        0  8.305141e-01      6
R-MMU-5654404.30.00 R-MMU-1169403        0  6.104673e-01      4
R-MMU-5654406.0.00  R-MMU-1169405        0  7.579585e-01      5
R-MMU-5654406.30.00 R-MMU-1169406        0  9.550000e-01     12
R-MMU-5654407.0.00  R-MMU-1678842        0  1.000000e+00     13
R-MMU-5654407.30.00 R-MMU-1678843        0  8.299585e-01      5
R-MMU-5654408.0.00  R-MMU-5653754       

R-MMU-5654659.30.00 R-MMU-2424487        0  9.597015e-01     22
R-MMU-5654662.0.00   R-MMU-388814        0  7.844030e-01      6
R-MMU-5654662.30.00  R-MMU-388829        0  1.000000e+00      9
R-MMU-5654663.0.00   R-MMU-388831        0  1.000000e+00      8
R-MMU-5654663.30.00  R-MMU-388832        0  1.000000e+00      9
R-MMU-5654664.0.00   R-MMU-388833        0  1.000000e+00     17
R-MMU-5654664.30.00  R-MMU-389158        0  1.000000e+00      9
R-MMU-5654667.0.00   R-MMU-389348        0  7.579585e-01      8
R-MMU-5654667.30.00  R-MMU-389350        0  8.024030e-01      8
R-MMU-5654669.0.00   R-MMU-389352        0  7.574030e-01      7
R-MMU-5654669.30.00  R-MMU-389354        0  7.574030e-01      7
R-MMU-5654672.0.00   R-MMU-389381        0  6.514515e-01      6
R-MMU-5654672.30.00  R-MMU-389758        0  9.597015e-01     22
R-MMU-5654673.0.00   R-MMU-389759        0  9.147015e-01      7
R-MMU-5654673.30.00  R-MMU-389762        0  8.299585e-01      5
R-MMU-5654677.0.00   R-MMU-434836       

R-MMU-9011082.30.00 R-MMU-9692139        0 -8.521242e-03      2
R-MMU-9011144.0.00  R-MMU-9692145        0  7.452156e-01      2
R-MMU-9011144.30.00 R-MMU-9692156        0  2.327625e-01      4
R-MMU-9011146.0.00  R-MMU-9692157        0  2.327625e-01      4
R-MMU-9011146.30.00 R-MMU-9692268        0  1.622031e-01      2
R-MMU-9753028.0.00   R-MMU-450434        0  3.275684e-01      1
R-MMU-9753028.30.00  R-MMU-450466        0  9.555556e-01     45
R-MMU-9753126.0.00   R-MMU-450551        0  9.111111e-01      7
R-MMU-9753126.30.00  R-MMU-450580        0  9.105556e-01     12
R-MMU-514604.0.00     R-MMU-72103        0  1.000000e+00     43
R-MMU-514604.30.00  R-MMU-9770119        0  1.000000e+00     69
R-MMU-975389.0.00   R-MMU-9770141        0  9.550000e-01    123
R-MMU-975389.30.00  R-MMU-9770142        0  1.000000e+00    132
R-MMU-1169394.0.00  R-MMU-9770145        0  1.000000e+00    145
R-MMU-1169394.30.00 R-MMU-9770236        0  9.222222e-01    119
R-MMU-1169398.0.00  R-MMU-6798748       

...

R-MMU-5682285.0.00  R-MMU-8849068        0  7.844030e-01      2
R-MMU-5682285.30.00 R-MMU-5694077        0  8.110000e-01      1
R-MMU-517536.0.00   R-MMU-9009434        0  7.118823e-01      2
R-MMU-517536.30.00  R-MMU-1181357        0  2.481002e-02      2
R-MMU-191983.0.00   R-MMU-6800149        0  7.632721e-02      2
R-MMU-191983.30.00  R-MMU-1675883        0  4.808383e-01      3
R-MMU-2161660.0.00  R-MMU-1676152        0  4.808383e-01      3
R-MMU-2161660.30.00 R-MMU-6798335        0  2.611652e-02      1
R-MMU-727740.0.00   R-MMU-9011543        0 -2.394844e-02      1
R-MMU-727740.30.00  R-MMU-6801101        0  4.651843e-01      2
R-MMU-727768.0.00   R-MMU-9692867        0  6.167641e-01      1
R-MMU-727768.30.00  R-MMU-9692880        0  6.167641e-01      1
R-MMU-1483096.0.00   R-MMU-444879        0  3.169832e-01      3
R-MMU-1483096.30.00  R-MMU-390929        0  6.124851e-01      3
R-MMU-1483159.0.00  R-MMU-9614271        0  1.226479e-01      1
R-MMU-1483159.30.00 R-MMU-9614273       

R-MMU-977348.0.00   R-MMU-1299338        0  6.387085e-01      1
R-MMU-977348.30.00  R-MMU-5694583        0  8.697015e-01      1
R-MMU-428961.0.00   R-NUL-9830947        0  4.977314e-01      1
R-MMU-428961.30.00  R-MMU-5423117        0  9.105556e-01     14
R-MMU-6807214.0.00  R-MMU-1296045        0  9.597015e-01      3
R-MMU-6807214.30.00  R-MMU-383190        0  8.379841e-03      1
R-MMU-177784.0.00   R-MMU-5678863        0  8.379841e-03      1
R-MMU-177784.30.00  R-MMU-8866851        0  8.379841e-03      1
R-MMU-71306.0.00     R-MMU-375776        0  7.956530e-01      1
R-MMU-71306.30.00   R-MMU-5218853        0  2.219354e-01      2
R-MMU-6797955.0.00  R-MMU-9607226        0  6.472870e-01      3
R-MMU-6797955.30.00   R-MMU-70881        0  5.675064e-01      1
R-MMU-70941.0.00     R-MMU-110224        0  2.444646e-01      1
R-MMU-70941.30.00    R-MMU-110226        0  2.444646e-01      1
R-MMU-374207.0.00    R-MMU-110227        0  2.444646e-01      1
R-MMU-374207.30.00   R-MMU-110229       

R-MMU-111898.0.00    R-MMU-374787        0  6.715272e-02      1
R-MMU-111898.30.00   R-MMU-373086        0  8.702570e-01      5
R-MMU-1482759.0.00   R-MMU-211178        0  1.203776e-01      1
R-MMU-1482759.30.00 R-MMU-9626928        0  1.203776e-01      1
R-MMU-6791016.0.00  R-NUL-9626944        0  1.203776e-01      1
R-MMU-6791016.30.00  R-MMU-170149        0  1.207597e-01      1
R-MMU-9624789.0.00  R-MMU-5671737        0  7.538126e-01      2
R-MMU-9624789.30.00 R-MMU-5671749        0  7.538126e-01      2
R-MMU-5358592.0.00  R-MMU-9029987        0  1.207597e-01      1
R-MMU-5358592.30.00 R-MMU-9733403        0  5.878939e-01      1
R-MMU-69173.0.00     R-MMU-419426        0  7.599259e-01      3
R-MMU-69173.30.00   R-MMU-9696323        0  7.599259e-01      3
R-MMU-9014847.0.00   R-MMU-374214        0  1.000000e+00      3
R-MMU-9014847.30.00  R-MMU-417898        0  9.152570e-01      1
R-MMU-3857329.0.00  R-MMU-8850786        0  6.167641e-01      1
R-MMU-3857329.30.00  R-MMU-400434       

R-MMU-2167942.0.00   R-MMU-417820        0  1.028430e-01      1
R-MMU-2167942.30.00 R-MMU-9707606        0  9.597015e-01      1
R-MMU-8944261.0.00  R-MMU-9707683        0  9.597015e-01      1
R-MMU-8944261.30.00 R-MMU-9707856        0  9.597015e-01      1
R-MMU-2465917.0.00  R-MMU-1483087        0  5.189515e-01      1
R-MMU-2465917.30.00 R-MMU-1483211        0  5.189515e-01      1
R-MMU-2466085.0.00  R-MMU-1483219        0  5.189515e-01      1
R-MMU-2466085.30.00 R-MMU-1483229        0  5.189515e-01      1
R-MMU-9656891.0.00  R-MMU-5693691        0  9.597015e-01      2
R-MMU-9656891.30.00 R-MMU-9749647        0 -1.503720e-02      1
R-MMU-3341294.0.00  R-MMU-9749792        0  9.597015e-01      2
R-MMU-3341294.30.00 R-MMU-5696119        0  6.688454e-01      1
R-MMU-976801.0.00   R-MMU-9673827        0  9.550000e-01      1
R-MMU-976801.30.00  R-MMU-2872444        0  1.705419e-01      1
R-MMU-9603568.0.00   R-MMU-204617        0  2.914839e-01      3
R-MMU-9603568.30.00  R-MMU-204647       

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [29]:
%%R
pid <- read.table(paste(OUT_DIR,"tcdd_project_id030_time_cross_70.txt",sep=""),header=FALSE)
sid <- read.table(paste(OUT_DIR,"tcdd_sample_id030_time_cross_70.txt",sep=""), header=FALSE)
print(sid)

           V1
1  SRR7817613
2  SRR7817695
3  SRR7817698
4  SRR1636593
5  SRR7817637
6  SRR7817686
7  SRR7817614
8  SRR7817639
9  SRR7817616
10 SRR7817696
11 SRR7817688
12 SRR3593527
13 SRR7817624
14 SRR7817659
15 SRR7817675
16 SRR3593525
17 SRR7817636
18 SRR7817697
19 SRR7817662
20 SRR4317608
21 SRR1636592
22 SRR7817664
23 SRR4317609
24 SRR7817661
25 SRR7817687
26 SRR7817684
27 SRR7817638
28 SRR1636591
29 SRR7817671
30 SRR7817627
31 SRR7817623
32 SRR7817674
33 SRR7817628
34 SRR7817640
35 SRR4317607
36 SRR1636586
37 SRR7817685
38 SRR4317612
39 SRR7817635
40 SRR1636590
41 SRR7817683
42 SRR7817672
43 SRR7817615
44 SRR7817626
45 SRR7817673
46 SRR7817652
47 SRR7817625
48 SRR3593579
49 SRR7817665
50 SRR3593574
51 SRR7817645
52 SRR7817706
53 SRR7817705
54 SRR7817658
55 SRR4317656
56 SRR7817655
57 SRR7817680
58 SRR7817689
59 SRR7817669
60 SRR7817617
61 SRR1636673
62 SRR7817630
63 SRR3593580
64 SRR3593573
65 SRR4317658
66 SRR7817643
67 SRR7817656
68 SRR7817654
69 SRR1636671
70 SRR7817646
71 SRR

In [ ]:
%%R
library(htmlwidgets)
min_misclass_pca <-
   prcomp(t(vst.count.mtx.train), scale. = T)
pca.d <- data.frame(
   PC1 = min_misclass_pca$x[, 1],
   PC2 = min_misclass_pca$x[, 2],
   PC3 = min_misclass_pca$x[, 3],
   Section = tcdd_dose_detail.vec.train,
   Project_id = pid$V1,
   Sample_id = sid$V1
)

ggplot(pca.d, aes(x = PC1, y = PC2, colour = Section)) +
   geom_point(size = 3) + 
   geom_text(aes(label = paste(Project_id, Sample_id)), vjust = -1, hjust = 1, size = 2) +  
   scale_colour_manual(values = c("0" = "red", "30" = "green")) +  
   theme_bw()

p <- plot_ly(
   data = pca.d,
   x = ~PC1,
   y = ~PC2,
   z = ~PC3,
   type = "scatter3d",
   mode = "markers",
   marker = list(
     color = as.factor(pca.d$Section), 
     colors = c("red", "green"),  
     size = 5
   ),
   text = ~paste("Project ID:", Project_id, "Sample ID:", Sample_id)  
)
saveWidget(p, "plotly_chart030_time_cross_70.html", selfcontained = TRUE)